<h2>Danh sách thành viên nhóm:</h2>
    <p>Trà Anh Toàn:            18120662</p>
    <p>Nguyễn Trần Nhật Minh:    18120208</p>
    <p>Nguyễn Vinh Quang:        18120229</p>
    <p>Đặng Văn Hiển:            18120000</p>
    <p>Lê Thanh Viễn:            18120666</p>

<h2>Bảng phân công công việc</h2>
<table>
<tr>
<td>Trà Anh Toàn </td>
<td>Nguyễn Trần Nhật Minh</td>
<td>Nguyễn Vinh Quang</td>
<td>Đặng Văn Hiển</td>
<td>Lê Thanh Viễn</td>
</tr>
<tr>
<td>Sử dụng api để lấy thông tin users</td>
<td>Tìm hiểu api, config, hoàn thiện báo cáo</td>
<td>Sử dụng api để lấy thông tin playlists, tracks</td>
<td>Crawl data lấy thông tin users, playlists, tracks</td>
<td>Tìm hiểu bố cục trang web, set up các thông tin cần thiết, lấy user url để crawl</td>
</tr>
</table>

<h2>Mô tả sơ lược thông tin dữ liệu</h2>
<h3>Số mẫu thu thập được</h3>
<table>
<tr>
<td>Users</td>
<td>Playlists</td>
<td>Tracks</td>
</tr>
<tr>
<td>1200</td>
<td>1900</td>
<td></td>
</tr>
    


Ý nghĩa các đặc trưng của bảng user :
*  username	: tên định danh 
*  permalink_url : đường dẫn trang cá nhân trân soundcloud
*  station_permalink : đường dẫn station 
*  city : thành phố 
*  country_code	: mã quốc gia
*  avatar_url	: đường dẫn hình đại diện
*  visual_url	: đường dẫn hình nền
*  first_name	: tên
*  last_name : họ
*  full_name : 
*  created_at	: ngày tạo
*  last_modified : ngày cập nhật gần nhất
*  creator_subscription : loại tài khoản
*  verified	: xác thực tài khoản nghệ sĩ
*  followers_count : số người theo dõi
*  followings_count	: số người user đang theo dõi
*  groups_count	: số group tham gia
*  likes_count : số like nhận được
*  comments_counts : số comment nhận được
*  playlist_likes_count : số like nhận được cho playlist 
*  track_count : số track
*  playlist_count : số playlist
*  reposts_count : số repost

Ý nghĩa các đặc trưng của bảng track :
* id : mã track
* title  : tiêu đề 
* description  : dòng miêu tả 
* duration : thời lượng 
* permalink_url : đường dẫn trang track
* genre : thể loại
* track_format  : định dạng
* artwork_url  : đường dẫn hình track
* tag_list  : danh sách các tag
* user_id  :  mã user sở hữu track
* station_permalink  : đường dẫn station
* album_title  : tiêu đề album
* waveform_url  : đường dẫn file waveform
* hls_url  : đường dẫn stream hls
* created_at  : ngày tạo
* last_modified  : ngày cập nhật gần nhất
* release_date  : ngày xuất bản
* display_date  : ngày hiển thị
* download_count  : số lượt tải về
* likes_count  : số like nhận được
* comment_count  : số bình luận nhận được
* reposts_count  : số lượt repost
* playback_count : số lần phát lại

Ý nghĩa các đặc trưng của bảng playlist :
* Id_user : mã user sở hữu playlist
* id : mã playlist
* title : tiêu đề 
* description : dòng miêu tả 
* duration : tổng thời lượng các track
* permalink_url : đường dẫn trang playlist
* genre : thể loại
* artwork_url : đường dẫn hình playlist
* tag_list : danh sách các tag
* created_at : ngày tạo
* last_modified : ngày cập nhật gần nhất
* release_date : ngày xuất bản
* display_date : ngày hiển thị
* tracks : danh sách các track (chuỗi đc tách nhau bởi dấu ',')
* track_count : số track trong playlist
* likes_count : số like nhận được
* reposts_count: số lượt repost
* is_album: là album hay không
* published_at: ngày phát hành


In [1]:
import sys
sys.executable
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import io
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian
# YOUR CODE HERE (OPTION) 
# Nếu cần các thư viện khác thì bạn có thể import ở đây

import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

In [2]:
def collect_id_playlist(id_user):
    client_id = '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn'
    url = f'https://api-v2.soundcloud.com/users/{id_user}/playlists?client_id={client_id}'
    #print(url)
    while(1):
        req = requests.get(url).json()
        if (len(req) != 0):
            req = req['collection'];
            id_playlist = ""
            dem = 0
            for k in req:
                for i in k:
                    if(i == "id"):
                        dem = dem + 1
                        #print(dem)
                        if(dem < 3):
                            id_playlist = id_playlist + str(k[i]) + ', '
                        elif(dem == 3):
                            id_playlist = id_playlist + str(k[i]) + '...'
                    if(dem == 3):
                        break
            return id_playlist
        break

In [3]:
def write_user_file_header(courses_file):
    url = 'https://api-v2.soundcloud.com/users/345785342?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn'
    req = requests.get(url).json()
    file = open(courses_file, 'w', encoding='utf-8')
    for i in req:
        if(i!="description"):
            file.write(f'{i}\t')
    file.write('playlists_Id\n')
    file.close()
    


In [4]:
def collect_user(id_user, courses_file):
    client_id = '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn'
    url = f'https://api-v2.soundcloud.com/users/{id_user}?client_id={client_id}'
    file = open(courses_file, 'a', encoding='utf-8')
    while(1):
        req = requests.get(url).json()
        #print(req)
        if (len(req) != 0):
            id = id_user
            for k in req:  
                if(k!="description"):
                    file.write(f'{req[k]}\t')
            
            playlists_id = collect_id_playlist(id)
            file.write(f'{playlists_id}\n')
            break

In [ ]:

            
client_id = '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn'
userIds = []
write_user_file_header('user.csv')
for i in range(0,1001,200):
    while (1):
        url = f'https://api-v2.soundcloud.com/search/users?q=1&client_id={client_id}&limit=200&offset={i}'
        req = requests.get(url).json()
        if ("collection" in req):
            req = req["collection"]
            for k in req:
                print(k["id"])
                collect_user(k["id"],'user.csv')
            break


            
    

47946400
76310082
367814642
67278526
88271741
96094926
137128168
102989060
19945124
122358203
161636
536616102
44365922
575612397
14691498
147999619
47793358
221660
1094882
459623
100926934
62638013
190748846
216160996
30214007
208518230
5747378
190316060
29131194
95251129
123600
792344461
14434100
305067533
63159176
3710978
147556137
148383503
162766233
78921656
231187185
642633195
185708230
798779
7722850
7073034
261376591
244964220
89958333
58292611
87153654
26130339
15061117
831318454
503813
222445497
138619288
225645479
51316637
124219184
283848085
326491730
345785342
61278382
9147872
236196469
57987099
95555437
4323076
342250558
73137584
220563576
441426276
37908345
159618390
31595440
334739
130645678
85867330
62908023
33156418
146809328
7955745
91422929
150276264
232136131
199757467
34715062
52923273
39494971
44088564
155080011
112149344
173312943
102842110
483316695
13461843
763500421
1328833
699360710
847184
174772052
32780128
264537238
1164187
222977229
151676870
93078933
207

In [5]:
def write_playlist_file_header(courses_file):
    url = 'https://api-v2.soundcloud.com/users/76310082/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn'
    
    file = open(courses_file, 'w', encoding='utf-8')
    file.write('Id_user\t')
    #print(req)
    finish = False
    while not finish:
        
        r = requests.get(url)
        if r.ok:
            req = r.json()
            req = req['collection'];
            for k in req:
                for i in k:  
                    if(i!="description"):
                        file.write(f'{i}\t')
                    if(i == "track_count"):
                        break
                break
            finish = True 
            
    file.write('tracks_Id\n')
    file.close()
    

In [6]:
write_playlist_file_header('playlist.csv')
courses = pd.read_csv('playlist.csv', sep='\t')
#courses

In [7]:
def collect_id_track(id_user):
    client_id = '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn'
    url = f'https://api-v2.soundcloud.com/users/{id_user}/playlists?client_id={client_id}'
    
    while (1):
        req = requests.get(url).json()
        if (len(req) != 0):
            
            req = req['collection'];
            id_tracks = ""
            for k in req:
                for i in k:
                    dem = 0
                    if(i == "tracks"):
                        for j in k[i]:
                            for m in j:
                                if(m == "id"):
                                    dem = dem + 1
                                    if (dem < 3):
                                        id_tracks = id_tracks + str(j[m]) + ', '
                                    elif(dem == 3):
                                        id_tracks = id_tracks + str(j[m]) + '...' 
                                        break
                        return id_tracks 
             


In [8]:
a = collect_id_track(76310082)
a

'1058454265, 1083282715, 1083292495...'

In [9]:
def collect_playlist(id_user, courses_file):
    client_id = '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn'
    url = f'https://api-v2.soundcloud.com/users/{id_user}/playlists?client_id={client_id}'
    file = open(courses_file, 'a', encoding='utf-8')
    #finish = False
    #while not finish:
    while (1):
        req = requests.get(url).json()
        if (len(req) != 0):
        
            req = req['collection'];
            id = id_user
            dem = 0
            for k in req:
                    
                for i in k:
                    if(i!="description"):
                            
                        if(i == "artwork_url"):
                            file.write(f'{id_user}\t')
                            file.write(f'{k[i]}\t')
                        if(i == "track_count"):
                            file.write(f'{k[i]}\t')
                            tracks_id = collect_id_track(id)
                            file.write(f'{tracks_id}\n')
                        else:
                            file.write(f'{k[i]}\t')
                    else:
                        dem = dem + 1
                    if(dem == 12):
                        break
            #finish = True
            break
            file.close()


In [10]:
write_playlist_file_header('playlist.csv')
collect_playlist(76310082, 'playlist.csv')
courses = pd.read_csv('playlist.csv', sep='\t')
courses

,Id_user,artwork_url,created_at,duration,embeddable_by,genre,id,kind,label_name,last_modified,...,uri,user_id,set_type,is_album,published_at,display_date,user,tracks,track_count,tracks_Id
76310082,https://i1.sndcdn.com/artworks-qBTJSLJuLbEqpsW...,https://i1.sndcdn.com/artworks-qBTJSLJuLbEqpsW...,2021-07-07T16:24:00Z,11908572,all,NaN,1283423632,playlist,None,2021-07-07T17:19:29Z,...,https://api.soundcloud.com/playlists/1283423632,76310082,NaN,False,2021-07-07T16:24:00Z,2021-07-07T16:24:00Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",9,"1058454265, 1083282715, 1083292495..."
76310082,None,None,2021-07-06T22:09:49Z,8094563,all,NaN,1283095417,playlist,None,2021-07-06T22:38:36Z,...,https://api.soundcloud.com/playlists/1283095417,76310082,NaN,False,2021-07-06T22:09:49Z,2021-07-06T22:09:49Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",9,"1058454265, 1083282715, 1083292495..."
76310082,None,None,2021-07-06T14:31:50Z,1670661,all,NaN,1282909864,playlist,None,2021-07-06T14:31:50Z,...,https://api.soundcloud.com/playlists/1282909864,76310082,NaN,False,2021-07-06T14:31:50Z,2021-07-06T14:31:50Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",1,"1058454265, 1083282715, 1083292495..."
76310082,None,None,2021-07-05T16:42:51Z,1410926,all,NaN,1282496320,playlist,None,2021-07-05T16:44:08Z,...,https://api.soundcloud.com/playlists/1282496320,76310082,NaN,False,2021-07-05T16:42:51Z,2021-07-05T16:42:51Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",1,"1058454265, 1083282715, 1083292495..."
76310082,None,None,2021-07-01T12:58:06Z,9833092,all,NaN,1280588587,playlist,None,2021-07-07T15:01:30Z,...,https://api.soundcloud.com/playlists/1280588587,76310082,NaN,False,2021-07-01T12:58:06Z,2021-07-01T12:58:06Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",14,"1058454265, 1083282715, 1083292495..."
76310082,None,None,2021-06-02T17:50:33Z,39668195,all,NaN,1266514681,playlist,None,2021-06-30T16:25:53Z,...,https://api.soundcloud.com/playlists/1266514681,76310082,NaN,False,2021-06-02T17:50:33Z,2021-06-02T17:50:33Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",56,"1058454265, 1083282715, 1083292495..."
76310082,None,None,2021-05-04T14:08:12Z,42375186,all,NaN,1252044148,playlist,None,2021-05-31T11:41:24Z,...,https://api.soundcloud.com/playlists/1252044148,76310082,NaN,False,2021-05-04T14:08:12Z,2021-05-04T14:08:12Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",57,"1058454265, 1083282715, 1083292495..."
76310082,None,None,2021-04-12T18:13:54Z,44598151,all,NaN,1241652682,playlist,None,2021-07-05T15:49:19Z,...,https://api.soundcloud.com/playlists/1241652682,76310082,NaN,False,2021-04-12T18:13:54Z,2021-04-12T18:13:54Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",13,"1058454265, 1083282715, 1083292495..."
76310082,None,None,2021-04-07T14:06:03Z,17214748,all,NaN,1238890684,playlist,None,2021-07-07T15:24:04Z,...,https://api.soundcloud.com/playlists/1238890684,76310082,NaN,False,2021-04-07T14:06:03Z,2021-04-07T14:06:03Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",12,"1058454265, 1083282715, 1083292495..."
76310082,None,None,2021-04-01T11:59:07Z,36200516,all,NaN,1235662162,playlist,None,2021-04-30T12:39:00Z,...,https://api.soundcloud.com/playlists/1235662162,76310082,NaN,False,2021-04-01T11:59:07Z,2021-04-01T11:59:07Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",47,"1058454265, 1083282715, 1083292495..."


In [ ]:
client_id = '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn'
userIds = []
#write_file_header('user.csv')
write_playlist_file_header('playlist.csv')
for i in (0,1001,200):
    
    while (1):
        url = f'https://api-v2.soundcloud.com/search/users?q=1&client_id={client_id}&limit=200&offset={i}'
        req = requests.get(url).json()
        if (len(req) != 0):
            if ("collection" in req):
                req = req["collection"]
                for k in req:
                    print(k["id"])
                    #collect_user(k["id"],'user.csv')
                    collect_playlist(k["id"], 'playlist.csv')
                break
            

47946400
76310082
367814642
67278526
88271741
96094926
137128168
102989060
19945124
122358203
161636
536616102
44365922
575612397
14691498
147999619
47793358
221660
1094882
459623
100926934
62638013
190748846
208518230
5747378
190316060
29131194
95251129
123600


In [ ]:
def write_track_file_header(courses_file):
    url = 'https://api-v2.soundcloud.com/users/76310082/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn'
    
    file = open(courses_file, 'w', encoding='utf-8')
    file.write('Id_user\t')
    file.write('Id_playlist\t')
    
    while(-1):
        req = requests.get(url).json()
        if(len(req) != 0):
            req = req['collection'];
            for k in req:
                for i in k:  
                    if(i!="tracks"):
                        j = k[i]
                        for m in j:
                            if(m !="description"):
                                file.write(f'{m}\t')
                            elif(m == 'policy'):
                                file.write(f'{m}\t')
                                break
        break
    file.close()
    

In [ ]:
def collect_track(id_user, courses_file):
    client_id = '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn'
    url = f'https://api-v2.soundcloud.com/users/{id_user}/playlists?client_id={client_id}'
    file = open(courses_file, 'w', encoding='utf-8')
    while(-1):
        req = requests.get(url).json()
        if (len(req) != 0):
            req = req['collection'];
            id = id_user
            file.write(f'{id}\t')
            for k in req:
                for i in k:
                    if(i == "id"):
                        id_playlist = k[i]
                        file.write(f'{id_playlist}\t')
                    elif(i == "tracks"):
                        dem = 0
                        exist = false
                        for j in k[i]:
                            for m in j:
                                if(m == "artwork_url"):
                                    exist = true
                                elif(m !="description"):
                                    file.write(f'{j[m]}\t')
                                elif(m == "description"):
                                    dem = dem + 1
                                elif(dem == 10):
                                    break
                                elif(exist == false):
                                    break
                                elif(m == "user"):
                                    exist = false
            break
            file.close()

In [ ]:
client_id = '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn'
userIds = []
#write_file_header('user.csv')
#write_playlist_file_header('playlist.csv')
write_track_file_header('track.csv')
for i in range(0,1001,200):
    while (-1):
        url = f'https://api-v2.soundcloud.com/search/users?q=1&client_id={client_id}&limit=200&offset={i}'
        req = requests.get(url).json()
        if ("collection" in req):
            req = req["collection"]
            for k in req:
                print(k["id"])
                #collect_user(k["id"],'user.csv')
                #collect_playlist(k["id"], 'playlist.csv')
                collect_track(k["id"], 'track.csv')
            break

In [ ]:
courses = pd.read_csv('track.csv', sep='\t')
courses


In [ ]:

courses = pd.read_csv('playlist.csv', sep='\t')
courses


In [ ]:

courses = pd.read_csv('user.csv', sep='\t')
courses